<a href="https://colab.research.google.com/github/melikebozkir/mashine_learning_classification/blob/main/vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd gdrive/My Drive/Colab Notebooks/yapay_zeka/derin_ogrenme/

/content/gdrive/My Drive/Colab Notebooks/yapay_zeka/derin_ogrenme


In [ ]:
%ls

basarili_model.h5  data/  deneme.png


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.layers import Activation, Dense



img_width, img_height = 400, 400

imageChannels = 3
batchSize = 1 #Her seferinde egitim icin alinacak veri miktari
epoch = 20
classMode= 'categorical'


fcDense = 3 #3 tane class
#Son katmandaki Sinif sayisi
#Binary (2 tane sinif) icin : fcDense = 1 olmali
fcActivation='softmax'
optimizer='adam'
loss= 'binary crossentropy'
metrics='accuracy'
train_data_yolu = 'data/train'
validation_data_yolu='data/validation'

train_ornek_sayisi = 3000
validation_ornek_sayisi = 1500

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [ ]:
input = Input (input_shape)
#Giris katmani Input modeli olarak uyarlaniyor
model= VGG16(include_top=False, weights='imagenet',input_tensor=input, classes=3)
#Modeling pooling ozellikleri ayarlanabilir: pooling-'avg'


# add new classifier layers
flat1= Flatten()(model.layers [-1].output)
class1= Dense(4096, activation='relu',name='fc1')(flat1)
class2 = Dense(4096, activation='relu',name='fc2')(class1)
output = Dense(fcDense, activation='softmax',name='predictions')(class2)
#define new model
model= Model(inputs=model.inputs, outputs=output)
model.summary()




58900480/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 400, 400, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 400, 400, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 400, 400, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 200, 200, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 200, 200, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 200, 200, 128)     147584    
                                                              

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
model.compile(
optimizer='adam',
loss='categorical_crossentropy',
metrics=['accuracy'],
)
#es = EarlyStopping (monitor='val_accuracy', mode='max', patience-5, restore_best_weights=True)

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 vertical_flip=True,rotation_range=45,
                                 fill_mode='nearest')
#Validation_split ayarlanarak subset alanlarina bolebilirsiniz subset="training", subset="validation"
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rescale=1. /255)


#Validation dataset 0.8*Training Set
train_generator = train_datagen.flow_from_directory(
train_data_yolu,
target_size=(img_width, img_height),
batch_size=batchSize,
#subset="training",
class_mode=classMode)


validation_generator = train_datagen.flow_from_directory(
train_data_yolu,
target_size=(img_width, img_height),
batch_size=batchSize,
#subset="validation",
class_mode=classMode)


#test_generator = test_datagen.flow_from_directory(
#validation_data_yolu,
#target_size=(img_width, img_height),
#batch_size=batchSize,
#class_mode=classMode)

Found 3000 images belonging to 3 classes.
Found 3000 images belonging to 3 classes.


In [ ]:
#Modelin Egitimi
model.fit(train_generator,
validation_data=validation_generator,
validation_steps=validation_ornek_sayisi, # test verisi/batchSize
#validation_steps=8//batchSize,
steps_per_epoch=train_ornek_sayisi, # egitim verisi/batchSize
#steps_per_epoch=20//batchSize, #egitim verisi/batchSize
batch_size=batchSize,
epochs=epoch,
verbose=1)

Epoch 1/20
3000/3000 [==============================] - 772s 253ms/step - loss: 13.8975 - accuracy: 0.3173 - val_loss: 1.0987 - val_accuracy: 0.3407
Epoch 2/20
3000/3000 [==============================] - 469s 156ms/step - loss: 1.0994 - accuracy: 0.3160 - val_loss: 1.0986 - val_accuracy: 0.3267
Epoch 3/20
3000/3000 [==============================] - 466s 155ms/step - loss: 1.0993 - accuracy: 0.3243 - val_loss: 1.0992 - val_accuracy: 0.3247
Epoch 4/20
3000/3000 [==============================] - 471s 157ms/step - loss: 1.0994 - accuracy: 0.3243 - val_loss: 1.0986 - val_accuracy: 0.3327
Epoch 5/20
3000/3000 [==============================] - 469s 156ms/step - loss: 1.0991 - accuracy: 0.3277 - val_loss: 1.0987 - val_accuracy: 0.3253
Epoch 6/20
3000/3000 [==============================] - 467s 156ms/step - loss: 1.0993 - accuracy: 0.3330 - val_loss: 1.0990 - val_accuracy: 0.3273
Epoch 7/20
3000/3000 [==============================] - 469s 156ms/step - loss: 9130.0342 - accuracy: 0.3260 - 

In [ ]:
model.save_weights('basarili_model_vgg.h5')

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np

model.load_weights('basarili_model_vgg.h5')

img = load_img('indir.jpg',grayscale=False, color_mode='rgb', target_size=(img_width, img_height))


x = img_to_array(img) #Numpy kutuphanesi ile bir diziye donusturuldu: shape(3, 224,224,3)
prediction = model.predict(x.reshape((1,img_width, img_height,3)),batch_size=32, verbose=0) #(1,224,224,3)
print(prediction)

[[0.328699   0.32871872 0.34258232]]
